# LLM Experiments

Early experiments with LLMs

## Libraries

In [1]:
import os
import re

import numpy as np
import pandas as pd

from kaggle.api.kaggle_api_extended import KaggleApi
from together import Together

# Easier display options for debugging: 

# Set the display width to a larger value
pd.set_option('display.width', 1000)

# Optionally, set the max column width to avoid truncating column data
pd.set_option('display.max_colwidth', None)

# Optionally, set the max number of columns to show all columns
pd.set_option('display.max_columns', None)

## Get test data

Initially downloaded the [Kaggle Sentiment140](https://www.kaggle.com/datasets/kazanova/sentiment140) dataset (a load of tweets with sentiment). But when I looked through those data the sentiment estimates seemed to be pretty bad. So now I'm using: the [Twitter Sentiment Analysis Dataset](https://www.kaggle.com/datasets/jp797498e/twitter-entity-sentiment-analysis) 


; useful for experimenting). Polarity is 0 for negative, 2 for neutral and 4 for positive.

In [2]:
data_dir = '../data/tweet_data'
#dataset = 'kazanova/sentiment140'
dataset = 'jp797498e/twitter-entity-sentiment-analysis'
# filename = training.1600000.processed.noemoticon.csv
filename = 'twitter_training.csv'


# Function to download dataset if not already downloaded
def download_kaggle_dataset(dataset, data_dir):
    if not os.path.exists(data_dir):
        os.makedirs(data_dir)
    
    # Check if dataset already exists
    dataset_files = os.listdir(data_dir)
    if not dataset_files:
        api = KaggleApi()
        api.authenticate()
        api.dataset_download_files(dataset, path=data_dir, unzip=True)
        print("Dataset downloaded and extracted.")
    else:
        print("Dataset already exists in the directory.")

# Run the function to download the dataset
download_kaggle_dataset(dataset, data_dir)


Dataset already exists in the directory.


In [3]:

#tweets_df = pd.read_csv(os.path.join(data_dir, filename),
#                        header=None,
#                        names=["polarity", "id", "date", "query", "user", "text"],
#                        dtype={"polarity": int, "id": int, "date": str, "query": str, "user": str, "text": str},
#                        encoding='latin1',
#                        index_col=False  # Ensure the df is given an ascending, consecutive index
#                        )

tweets_df = pd.read_csv(os.path.join(data_dir, filename),
                        header=None,
                        names=["x", "entity", "sentiment", "text"],
                        dtype={"x": int, "entity": str, "sentiment": str, "text": str},
                        index_col=False  # Ensure the df is given an ascending, consecutive index
                        )

tweets_df

x       entity sentiment                                                                                                                                                             text
0      2401  Borderlands  Positive                                                                                                            im getting on borderlands and i will murder you all ,
1      2401  Borderlands  Positive                                                                                                              I am coming to the borders and I will kill you all,
2      2401  Borderlands  Positive                                                                                                               im getting on borderlands and i will kill you all,
3      2401  Borderlands  Positive                                                                                                              im coming on borderlands and i will murder you all,
4      2401  Borderlands  Positive                                                                                                        im getting on borderlands 2 and i will murder you me all,
...     ...          ...       ...                                                                                                                                                              ...
74677  9200       Nvidia  Positive                                 Just realized that the Windows partition of my Mac is like 6 years behind Nvidia drivers and I have no idea how I did not notice
74678  9200       Nvidia  Positive                                            Just realized that my Mac window partition is 6 years behind on Nvidia drivers and I have no idea how I didn't notice
74679  9200       Nvidia  Positive                                    Just realized the windows partition of my Mac is now 6 years behind on Nvidia drivers and I have no idea how he didn’t notice
74680  9200       Nvidia  Positive  Just realized between the windows partition of my Mac is like being 6 years behind on Nvidia drivers and cars I have no fucking idea how I ever didn ’ t notice
74681  9200       Nvidia  Positive                                          Just like the windows partition of my Mac is like 6 years behind on its drivers So you have no idea how I didn’t notice

[74682 rows x 4 columns]

## Use the Together.AI API to batch classify them. 

In [4]:
# Get the API key from a file
with open('together.ai_key.txt', 'r') as f:
    api_key = f.readline().strip()

client = Together(api_key=api_key)

# Drop the 'irrelevant' sentiment
tweets_df = tweets_df[tweets_df.sentiment != 'Irrelevant']
# List of tweets to classify (only a few for now)
df = tweets_df.sample(200).copy()
# Ensure the index is consecutive and ascending
df = df.reset_index(drop=True)
# To store the results
df['sentiment_prediction'] = np.nan

# Whole thing needs to be nested in a for loop batches the tweets
batch_size = 20 
for i in range(0, len(df), batch_size):
    # Get the list of tweets in this batch
    batch_tweets = df.loc[i:i+batch_size,:]
    tweet_list = "\n".join([f"{idx+1}. {tweet}" 
                            for idx, tweet in enumerate(batch_tweets.text.values)])
    #print(tweet_list)

    # Create the system prompt
    system_prompt = (
        "Classify the sentiment (positive, negative, or neutral) of each of the following texts. "
        "Provide your answer in the format '1. Sentiment', '2. Sentiment', etc.\n\n"
        f"{tweet_list}"
    )

    # Prepare the messages
    messages = [
        {
            "role": "system",
            "content": system_prompt
        }
    ]
    
    print(f"Submitting batch {i//batch_size+1} of {len(df)//batch_size}...",)
    # Call the API (docs here: https://docs.together.ai/reference/chat-completions-1)
    response = client.chat.completions.create(
        model="meta-llama/Llama-3.2-90B-Vision-Instruct-Turbo",
        messages=messages,
        max_tokens=300,
        temperature=0.7,
        top_p=0.7,
        top_k=50,
        repetition_penalty=1,
        stop=["<|eot_id|>", "<|eom_id|>"],
        truncate=130560,
        stream=False  # Set stream to False to get the full response
    )

    # Extract the assistant's reply
    assistant_reply = response.choices[0].message.content.strip()
    print("... reply received.")

    ## Print the assistant's reply
    #print("Assistant's Reply:")
    #print(assistant_reply)

    # Use regular expressions to extract the sentiments
    matches = re.findall(r"(\d+)\.\s*(Positive|Negative|Neutral)", assistant_reply, re.IGNORECASE)

    # Check that the numbering is correct
    # TODO
    
    # Put those estimates back into the df via two lists (one for the index, one for the sentiment)
    # (remember to convert the sentiment id back to the dataframe index)
    ids = [i+(int(idx)-1)  for (idx, sentiment) in matches]
    sentiments = [sentiment.capitalize() for (idx, sentiment) in matches]
    assert len(ids) == len(sentiments)
    
    # Put those estimates back into the df
    df.loc[ids, 'sentiment_prediction'] = sentiments
    

    ## Print the results
    #for idx, tweet in enumerate(tweets):
    #    sentiment = sentiments.get(idx, "Not Classified")
    #    print(f"Tweet: {tweet}")
    #    print(f"Sentiment: {sentiment}")
    #    print()

Submitting batch 1 of 10...
... reply received.
Submitting batch 2 of 10...
... reply received.
Submitting batch 3 of 10...
... reply received.
Submitting batch 4 of 10...
... reply received.
Submitting batch 5 of 10...
... reply received.
Submitting batch 6 of 10...
... reply received.
Submitting batch 7 of 10...
... reply received.
Submitting batch 8 of 10...
... reply received.
Submitting batch 9 of 10...
... reply received.
Submitting batch 10 of 10...
... reply received.


In [5]:
#df = df.loc[df.sentiment != 'Irrelevant']
df

x                  entity sentiment                                                                                                                                                                                                                                                                            text sentiment_prediction
0     9445               Overwatch  Negative                                                                                                                                                 Sure would be nice if you actually cared about their game instead of just trying to kill it with one bad game after the second.             Negative
1    11875                 Verizon  Negative                                                                                                                                                                                                                      Verizon full of shit I lose my service somewhere every day             Negative
2     5561             Hearthstone  Positive                                                                                                                                                                                                                                                            Thank you Blizzard..             Positive
3    12710            WorldOfCraft   Neutral                                                                                                                                                             World by Motion: New Release Date Leaked - GS<unk> Commons zonestream.cx/?p=2924&feed_i... https://t.co/raZS1VkoHy]              Neutral
4     8653                   NBA2K  Positive                                                                                                                                                                                         @ NBA2K _ MyTEAM Jamal is my favorite player, depending on if he deserves an upgrade!!!             Positive
..     ...                     ...       ...                                                                                                                                                                                                                                                                             ...                  ...
195   6259                    FIFA  Negative                                                                                                                                                                                                                                                         So sad to see Fifa fall             Negative
196   2013              CallOfDuty  Positive                                                                                                                                                                                                                                                         Great day salute sir...             Positive
197   8593                   NBA2K  Negative                                                                                                                                                                                                                                    @Ronnie2K @NBA2K your game is dirty. Fix it.             Negative
198   2282              CallOfDuty  Negative                                                                                                                                                                                 @CallofDuty is ass, please fix that damn game.  Why does my gun turn into a sea urchin of lava?             Negative
199  10512  RedDeadRedemption(RDR)  Positive  @ RockstarGames I never played the game Red Dead Redeemption until last week because the Western thing didn't really appeal to me... But heck, I know I'll play every game now if it's not the last!! That's how the game is done, and that's why it 

See how well that worked

In [6]:
def check_sentiment_estimate(row):
    if row.polarity == 0 and row.sentiment == "Negative":
        return 1
    elif row.polarity == 2 and row.sentiment == "Neutral":
        return 1
    elif row.polarity == 4 and row.sentiment == "Positive":
        return 1
    else:
        return 0
    
def check_sentiment_estimate2(row):
    if row.sentiment == row.sentiment_prediction:
        return 1
    elif row.sentiment == "Irrelevant" and row.sentiment_prediction == "Neutral":
        return 1
    else:
        return 0

df['correct'] = df.apply(check_sentiment_estimate2, axis=1)
print("Accuracy:", df.correct.mean())
df.loc[:,['text', 'sentiment', 'sentiment_prediction', 'correct']]


Accuracy: 0.705


text sentiment sentiment_prediction  correct
0                                                                                                                                                   Sure would be nice if you actually cared about their game instead of just trying to kill it with one bad game after the second.  Negative             Negative        1
1                                                                                                                                                                                                                        Verizon full of shit I lose my service somewhere every day  Negative             Negative        1
2                                                                                                                                                                                                                                                              Thank you Blizzard..  Positive             Positive        1
3                                                                                                                                                               World by Motion: New Release Date Leaked - GS<unk> Commons zonestream.cx/?p=2924&feed_i... https://t.co/raZS1VkoHy]   Neutral              Neutral        1
4                                                                                                                                                                                           @ NBA2K _ MyTEAM Jamal is my favorite player, depending on if he deserves an upgrade!!!  Positive             Positive        1
..                                                                                                                                                                                                                                                                              ...       ...                  ...      ...
195                                                                                                                                                                                                                                                         So sad to see Fifa fall  Negative             Negative        1
196                                                                                                                                                                                                                                                         Great day salute sir...  Positive             Positive        1
197                                                                                                                                                                                                                                    @Ronnie2K @NBA2K your game is dirty. Fix it.  Negative             Negative        1
198                                                                                                                                                                                 @CallofDuty is ass, please fix that damn game.  Why does my gun turn into a sea urchin of lava?  Negative             Negative        1
199  @ RockstarGames I never played the game Red Dead Redeemption until last week because the Western thing didn't really appeal to me... But heck, I know I'll play every game now if it's not the last!! That's how the game is done, and that's why it has the highest rating...  Positive                  NaN        0

[200 rows x 4 columns]